In [1]:
import tkinter as tk
from tkinter import Message ,Text
import cv2,os
import shutil
import csv
import numpy as np
from PIL import Image, ImageTk
import pandas as pd
import datetime
import time
import tkinter.ttk as ttk
import tkinter.font as font

window = tk.Tk()
window.title('Attendance Portal CTAE')

dialog_title = 'QUIT'
dialog_text = 'Are you sure?'

window.configure(background='sky blue')
window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)

message = tk.Label(window, text="Face Recognition based Attendance Portal for CTAE" ,bg="steel blue"  ,fg="white"  ,width=57  ,height=3,font=('times', 30, 'bold')) 
message.place(relx=0.5, y=10, anchor='n')

lbl = tk.Label(window, text="Enter Enrollment No. :",width=20  ,height=2  ,fg="midnight blue"  ,bg="sky blue" ,font=('Helvetica', 17, ' bold ') ) 
lbl.place(x=200, y=190)

txt = tk.Entry(window,width=20  ,bg="white" ,fg="midnight blue",font=('Helvetica', 15, ' bold '))
txt.place(x=550, y=205)

lbl2 = tk.Label(window, text="Enter Name :",width=20  ,fg="midnight blue"  ,bg="sky blue"    ,height=2 ,font=('Helvetica', 17, ' bold ')) 
lbl2.place(x=148, y=270)

txt2 = tk.Entry(window,width=20  ,bg="white"  ,fg="midnight blue",font=('Helvetica', 15, ' bold ')  )
txt2.place(x=550, y=285)

lbl3 = tk.Label(window, text="Notification : ",width=20  ,fg="midnight blue"  ,bg="sky blue"  ,height=2 ,font=('Helvetica', 17, ' bold ')) 
lbl3.place(x=150, y=370)

message = tk.Label(window, text="" ,bg="white"  ,fg="midnight blue"  ,width=45  ,height=2, activebackground = "white" ,font=('Helvetica', 15, ' bold ')) 
message.place(x=550, y=370)

lbl3 = tk.Label(window, text="Attendance : ",width=20  ,fg="midnight blue"  ,bg="sky blue"  ,height=2 ,font=('Helvetica', 17, ' bold ')) 
lbl3.place(x=141, y=569)

message2 = tk.Label(window, text="" ,fg="midnight blue"   ,bg="white",activeforeground = "green",width=35  ,height=2  ,font=('helvetica', 15, ' bold ')) 
message2.place(x=550, y=569)
 
def clear():
    txt.delete(0, 'end')    
    res = ""
    message.configure(text= res)

def clear2():
    txt2.delete(0, 'end')    
    res = ""
    message.configure(text= res)    
    
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False
 
def TakeImages():        
    Id=(txt.get())
    name=(txt2.get())
    if(is_number(Id) and name.isalpha()):
        cam = cv2.VideoCapture(0)
        harcascadePath = "haarcascade_frontalface_default.xml"
        detector=cv2.CascadeClassifier(harcascadePath)
        sampleNum=0
        while(True):
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)        
                #incrementing sample number 
                sampleNum=sampleNum+1
                #saving the captured face in the dataset folder TrainingImage
                cv2.imwrite("TrainingImage/"+name +"."+Id +'.'+ str(sampleNum) + ".jpg", gray[y:y+h,x:x+w])
                #display the frame
                cv2.imshow('frame',img)
            #wait for 100 miliseconds 
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
            # break if the sample number is morethan 100
            elif sampleNum>60:
                break
        cam.release()
        cv2.destroyAllWindows() 
        res = "Images Saved for ID : " + Id +" Name : "+ name
        row = [Id , name]
        with open('StudentDetails/StudentDetails.csv','a+') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        csvFile.close()
        message.configure(text= res)
    else:
        if(is_number(Id)):
            res = "Enter Alphabetical Name"
            message.configure(text= res)
        if(name.isalpha()):
            res = "Enter Numeric Id"
            message.configure(text= res)
    
def TrainImages():
    recognizer = cv2.face_LBPHFaceRecognizer.create()#recognizer = cv2.face.LBPHFaceRecognizer_create()#$cv2.createLBPHFaceRecognizer()
    harcascadePath = "haarcascade_frontalface_default.xml"
    detector =cv2.CascadeClassifier(harcascadePath)
    faces,Id = getImagesAndLabels("TrainingImage")
    recognizer.train(faces, np.array(Id))
    recognizer.save("TrainingImage/Trainner.yml")
    res = "Image Trained"#+",".join(str(f) for f in Id)
    message.configure(text= res)

def getImagesAndLabels(path):
    #get the path of all the files in the folder
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    #print(imagePaths)
    
    #create empth face list
    faces=[]
    #create empty ID list
    Ids=[]
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces.append(imageNp)
        Ids.append(Id)        
    return faces,Ids

def TrackImages():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    #cv2.createLBPHFaceRecognizer()
    recognizer.read("TrainingImage/Trainner.yml")
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath);    
    df=pd.read_csv("StudentDetails/StudentDetails.csv")
    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX        
    col_names =  ['Id','Name','Date','Time']
    attendance = pd.DataFrame(columns = col_names)    
    while True:
        ret, im =cam.read()
        gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        faces=faceCascade.detectMultiScale(gray, 1.2,5)    
        for(x,y,w,h) in faces:
            cv2.rectangle(im,(x,y),(x+w,y+h),(225,0,0),2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])                                   
            if(conf < 50):
                ts = time.time()      
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa=df.loc[df['Id'] == Id]['Name'].values
                tt=str(Id)+"-"+aa
                attendance.loc[len(attendance)] = [Id,aa,date,timeStamp]
                
            else:
                Id='Unknown'                
                tt=str(Id)  
            if(conf > 75):
                noOfFile=len(os.listdir("ImagesUnknown"))+1
                cv2.imwrite("ImagesUnknown/Image"+str(noOfFile) + ".jpg", im[y:y+h,x:x+w])            
            cv2.putText(im,str(tt),(x,y+h), font, 1,(255,255,255),2)        
        attendance=attendance.drop_duplicates(subset=['Id'],keep='first')    
        cv2.imshow('im',im) 
        if (cv2.waitKey(1)==ord('q')):
            break
    ts = time.time()      
    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
    Hour,Minute,Second=timeStamp.split(":")
    fileName="Attendance/Attendance_"+date+"_"+Hour+"-"+Minute+"-"+Second+".csv"
    attendance.to_csv(fileName,index=False)
    cam.release()
    cv2.destroyAllWindows()
    #print(attendance)
    res=attendance
    message2.configure(text= res)

  
clearButton = tk.Button(window, text="Clear", command=clear  ,fg="white"  ,bg="steel blue"  ,width=10  ,height=1 ,activebackground = "white" ,font=('Helvetica', 15, ' bold '))
clearButton.place(x=950, y=200)
clearButton2 = tk.Button(window, text="Clear", command=clear2  ,fg="white"  ,bg="steel blue"  ,width=10  ,height=1, activebackground = "white" ,font=('Helvetica', 15, ' bold '))
clearButton2.place(x=950, y=275)    
takeImg = tk.Button(window, text="Take Images", command=TakeImages  ,fg="white"  ,bg="steel blue"  ,width=15  ,height=2, activebackground = "Red" ,font=('Helvetica', 15, ' bold '))
takeImg.place(x=100, y=470)
trainImg = tk.Button(window, text="Train Images", command=TrainImages  ,fg="white"  ,bg="steel blue"  ,width=15  ,height=2, activebackground = "Red" ,font=('Helvetica', 15, ' bold '))
trainImg.place(x=400, y=470)
trackImg = tk.Button(window, text="Track Images", command=TrackImages  ,fg="white"  ,bg="steel blue"  ,width=15  ,height=2, activebackground = "Red" ,font=('Helvetica', 15, ' bold '))
trackImg.place(x=700, y=470)
quitWindow = tk.Button(window, text="Quit", command=window.destroy  ,fg="white"  ,bg="steel blue"  ,width=15  ,height=2, activebackground = "Red" ,font=('Helvetica', 15, ' bold '))
quitWindow.place(x=1000, y=470)
 
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Priyanka\Anaconda2\lib\lib-tk\Tkinter.py", line 1541, in __call__
    return self.func(*args)
  File "<ipython-input-1-4bea7d050e0d>", line 87, in TakeImages
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

